In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
from optuna import Trial
from typing import Dict, Union, Any
# import os
# import sys
# notebook_path = os.path.abspath('')

# # Find the part of the path that contains 'commitFit'
# commit_fit_path = None
# for part in notebook_path.split(os.sep):
#     print(part)
#     if 'CommitFit' in part:
#         commit_fit_path = notebook_path.split(part)[0] + part
#         break

# if commit_fit_path is None:
#     raise ValueError("Path containing 'commitFit' not found in notebook path.")

# # Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
# if commit_fit_path not in sys.path:
#     sys.path.append(commit_fit_path)
# from evaluate.metrics import accuracy,precision,recall,f1
# CUDA_LAUNCH_BLOCKING=1
import evaluate

In [2]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_1518/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_1518/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [3]:
train

,label,text,diff
0,0,Fixed link errors,diff --git a/src/irisnet/CMakeLists.txt b/src/...
1,1,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
2,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,1,XSS 취약점 수정,diff --git a/adm/boardgroup_form.php b/adm/boa...
4,0,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...
5,0,Merge pull request #2426 from alvarobartt/deve...,diff --git a/nltk/tag/stanford.py b/nltk/tag/s...
6,0,2021 license,diff --git a/Gruntfile.js b/Gruntfile.js\ninde...
7,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
8,0,Upgrade: Bump grape from 1.3.3 to 1.5.0\n\nBum...,diff --git a/Gemfile.lock b/Gemfile.lock\ninde...
9,0,Merge branch '8.5' into 9.5,diff --git a/src/Framework/Assert.php b/src/Fr...


In [4]:
len(train)

10

In [5]:
train['label'].value_counts()

label
0    6
1    4
Name: count, dtype: int64

In [6]:
test['label'].value_counts()

label
0    6341
1    3761
Name: count, dtype: int64

In [7]:
from sklearn import  metrics

# def compute_metrics(y_pred, y_test):
#     # print(y_pred,y_test)
#     # classification_report = metrics.classification_report(y_test,y_pred)
#     accuracy_score = metrics.accuracy_score(y_test,y_pred)
#     recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
#     f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

#     # return {"classification_report": classification_report}
#     return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [8]:
train

,label,text,diff
0,0,Fixed link errors,diff --git a/src/irisnet/CMakeLists.txt b/src/...
1,1,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
2,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,1,XSS 취약점 수정,diff --git a/adm/boardgroup_form.php b/adm/boa...
4,0,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...
5,0,Merge pull request #2426 from alvarobartt/deve...,diff --git a/nltk/tag/stanford.py b/nltk/tag/s...
6,0,2021 license,diff --git a/Gruntfile.js b/Gruntfile.js\ninde...
7,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
8,0,Upgrade: Bump grape from 1.3.3 to 1.5.0\n\nBum...,diff --git a/Gemfile.lock b/Gemfile.lock\ninde...
9,0,Merge branch '8.5' into 9.5,diff --git a/src/Framework/Assert.php b/src/Fr...


In [9]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "../../bert-base-cased", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained('../../bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [13]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [15]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load('../../evaluate/metrics/accuracy')
    precision =  evaluate.load("../../evaluate/metrics/precision")
    recall =  evaluate.load("../../evaluate/metrics/recall")
    f1 =  evaluate.load("../../evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')

    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [16]:
# !export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000
# !export https_proxy=http://turbo2.gpushare.com:30000 http_proxy=http://turbo2.gpushare.com:30000

In [17]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> model:
    # params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return model
def my_compute_objective(metrics):
    print('+++++++++++',metrics['eval_accuracy']['accuracy'])
    return  metrics['eval_accuracy']['accuracy']

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    model_init= model_init,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:423: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
[I 2024-05-29 15:18:16,316] A new study created in memory with name: no-name-bcab679c-e4bb-4da1-b708-a70077795ca9
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_1518/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than to

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.659024,{'precision': 0.39395785757550217},{'recall': 0.6274995050485053},{'f1': 0.4840306990465996},{'accuracy': 0.6274995050485053}


+++++++++++ 0.6274995050485053


[I 2024-05-29 15:18:53,708] Trial 0 finished with value: 0.6274995050485053 and parameters: {'learning_rate': 3.806916486028963e-05}. Best is trial 0 with value: 0.6274995050485053.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_1518/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.657486,{'precision': 0.5554191444877937},{'recall': 0.6262126311621461},{'f1': 0.49180286897911657},{'accuracy': 0.6262126311621461}


...

+++++++++++ 0.5089091269055632


[I 2024-05-29 15:23:42,040] Trial 8 pruned. 
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_1518/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.200225,{'precision': 0.6722061227849545},{'recall': 0.4930706790734508},{'f1': 0.4500965736170964},{'accuracy': 0.4930706790734508}


+++++++++++ 0.4930706790734508


[I 2024-05-29 15:24:16,788] Trial 9 pruned. 


In [19]:
best_run

BestRun(run_id='0', objective=0.6274995050485053, hyperparameters={'learning_rate': 3.806916486028963e-05}, run_summary=None)